In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from util import show, plot_images, plot_tensors

In [ ]:

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import Dataset
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms
from skimage.measure import compare_psnr

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


EPOCHS = 2
BATCH_SIZE = 20
LEARNING_RATE = 0.003
TRAIN_DATA_PATH = "./Confocal_BPAE_B/raw"
GT_DATA_PATH = "./Confocal_BPAE_B/gt"
TRANSFORM_IMG = transforms.Compose([
    transforms.CenterCrop(64),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
    
    ])
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
gt_data = torchvision.datasets.ImageFolder(root=GT_DATA_PATH, transform=TRANSFORM_IMG)


In [ ]:
dataset1 = train_data
dataset2 = gt_data


class gtmatch(Dataset):
    def __init__(self, dataset1, dataset2, batch_size):
        self.dataset1 = dataset1
        self.dataset2 = dataset2
        
    def __getitem__(self, index):
        x = self.dataset1[index][0]
        y= self.dataset2[int(index/batch_size % batch_size)][0]
        return x, y
    
    def __len__(self):
        return len(self.dataset1)

newdataset = gtmatch(dataset1, dataset2, 50)

In [ ]:
batch_size=50
from torch import randn
def add_noise(img):
    return img + randn(img.size())*0.05

class SyntheticNoiseDataset(Dataset):
    def __init__(self, data, mode='train'):
        self.mode = mode
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img = self.data[index][0]
        return add_noise(img), img



In [ ]:
noisy_data_train = SyntheticNoiseDataset(train_data, 'train')

In [ ]:
noisy,clean=newdataset[61]
plot_tensors([noisy[0], clean[0]], ['Noisy Image', 'Clean Image'])

In [ ]:
from mask import Masker
masker = Masker(width = 4, mode='interpolate')
net_input, mask = masker.mask(noisy.unsqueeze(0), 3)
plot_tensors([mask, noisy[0], net_input[0], net_input[0] - noisy[0]],
            ["Mask", "Noisy Image", "Neural Net Input", "Difference"])

In [ ]:
from models.babyunet import BabyUnet

model = BabyUnet()

In [ ]:
from torch.nn import MSELoss
from torch.optim import Adam
from torch.utils.data import DataLoader
import logging 
loss_function = MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [ ]:
n=800
t=1000
split=torch.utils.data.random_split(newdataset, [n,t-n]);
train=split[0];
test=split[1];

In [ ]:
from mask import Masker

data_loader = DataLoader(train, batch_size=50, shuffle=True)

#noisy, clean = noisy_data_train[0];
#net_input, mask = masker.mask(noisy.unsqueeze(0), 0);

losses = []
val_losses = []
best_images = []
best_val_loss = 1

for i, batch in enumerate(data_loader):
    noisy_images, clean_images = batch
    
    model.train()
 
    net_input, mask = masker.mask(noisy_images, i)
    net_output = model(net_input)
    
    loss = loss_function(net_output*mask, noisy_images*mask)
    
    optimizer.zero_grad()
 
    loss.backward()
    
    optimizer.step()

    losses.append(loss.item())
    model.eval()
    

    net_input, mask = masker.mask(noisy_images, masker.n_masks - 1)
    net_output = model(net_input)

    val_loss = loss_function(net_output*mask, noisy_images*mask)

    val_losses.append(val_loss.item())



    print("Loss (", i, "): \t", round(loss.item(), 4))
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        denoised = np.clip(model(noisy_images).detach().cpu().numpy()[0, 0], 0, 1).astype(np.float64)
        clean=np.clip(clean_images.detach().cpu().numpy()[0, 0], 0, 1).astype(np.float64)
        best_psnr = compare_psnr(denoised, clean)
        best_images.append(denoised)
        print("\tModel PSNR: ", np.round(best_psnr, 2))



In [ ]:
test_data_loader = DataLoader(test,
                                              batch_size=50,
                                              shuffle=True,
                                              num_workers=3)
i, test_batch = next(enumerate(test_data_loader))
noisy, clean = test_batch

In [ ]:
simple_output = model(noisy)
invariant_output = masker.infer_full_image(noisy, model)

In [ ]:
idx = 19
plot_tensors([clean[idx], noisy[idx], simple_output[idx], invariant_output[idx]],
            ["Ground Truth", "Noisy Image", "Single Pass Inference", "J-Invariant Inference"])

In [ ]:
print("Test loss, single pass: ", round(loss_function(clean, simple_output).item(), 3))
print("Test loss, J-invariant: ", round(loss_function(clean, invariant_output).item(), 3))

In [1]:
import os
git_dir = "{}/{}".format(os.path.expandvars(os.environ.get('~/Notebooks')), os.path.expandvars(os.environ.get('Deep-Learning-Denoising')))

In [2]:
git_dir

'~/Notebooks/Deep-Learning-Denoising'

In [31]:
os.path.expandvars(os.environ.get("~/Notebooks"))

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [9]:
import os

In [11]:
print(os.getenv('GIT_PARENT_DIR'))

/wynton/home/huang/emaad/Notebooks


In [20]:
print(os.getenv('/wynton/home/huang/emaad/Norebooks'))

None


In [12]:
def put(self):

    # git parameters from environment variables

    git_dir = "{}/{}".format(os.path.expandvars(os.environ.set('GIT_PARENT_DIR')), os.path.expandvars(os.environ.set('GIT_REPO_NAME')))
    git_url = os.path.expandvars(os.environ.get('GIT_REMOTE_URL'))
    git_user = os.path.expandvars(os.environ.get('GIT_USER'))
    git_repo_upstream = os.path.expandvars(os.environ.get('GIT_REMOTE_UPSTREAM'))
    git_branch = git_remote = os.path.expandvars(os.environ.get('GIT_BRANCH_NAME'))
    git_access_token = os.path.expandvars(os.environ.get('GITHUB_ACCESS_TOKEN'))

In [37]:
a="1" + "/" + "1"

In [38]:
a

'1/1'

In [23]:
git_dir = "{}/{}".format(os.path.expandvars(os.environ.get('GIT_PARENT_DIR')), os.path.expandvars(os.environ.get('GIT_REPO_NAME')))
git_url = os.path.expandvars(os.environ.get('GIT_REMOTE_URL'))
git_user = os.path.expandvars(os.environ.get('GIT_USER'))
git_repo_upstream = os.path.expandvars(os.environ.get('GIT_REMOTE_UPSTREAM'))
git_branch = git_remote = os.path.expandvars(os.environ.get('GIT_BRANCH_NAME'))
git_access_token = os.path.expandvars(os.environ.get('GITHUB_ACCESS_TOKEN'))

In [26]:
git_branch

'master'

In [27]:
os.path.dirname(git_dir)

'/wynton/home/huang/emaad/Notebooks'

In [28]:
filename

NameError: name 'filename' is not defined

In [29]:
os.environ.get('GIT_PARENT_DIR') + "/" + os.environ.get('GIT_REPO_NAME') 

'/wynton/home/huang/emaad/Notebooks/Deep-Learning-Denoising'

In [32]:
# expand variables since Docker's will pass VAR=$VAL as $VAL without expansion
git_dir = "{}/{}".format(os.path.expandvars('GIT_PARENT_DIR'), os.path.expandvars('GIT_REPO_NAME'))
git_url = os.path.expandvars(os.environ.get('GIT_REMOTE_URL'))
git_user = os.path.expandvars(os.environ.get('GIT_USER'))
git_repo_upstream = os.path.expandvars(os.environ.get('GIT_REMOTE_UPSTREAM'))
git_branch = git_remote = os.path.expandvars(os.environ.get('GIT_BRANCH_NAME'))
git_access_token = os.path.expandvars(os.environ.get('GITHUB_ACCESS_TOKEN'))

# get the parent directory for git operations
git_dir_parent = os.path.dirname(git_dir)

# obtain filename and msg for commit
data = json.loads(self.request.body.decode('utf-8'))
filename = urllib.parse.unquote(data['filename'])
msg = data['msg']
commit_only_source = data['commit_only_source']


# get current directory (to return later)
cwd = os.getcwd()

# select branch within repo
try:
    os.chdir(git_dir)
    dir_repo = check_output(['git','rev-parse','--show-toplevel']).strip()
    repo = Repo(dir_repo.decode('utf8'))
except GitCommandError as e:
    self.error_and_return(cwd, "Could not checkout repo: {}".format(dir_repo))
    return

# create new branch
try:
    print(repo.git.checkout('HEAD', b=git_branch))
except GitCommandError:
    print("Switching to {}".format(repo.heads[git_branch].checkout()))

# commit current notebook
# client will sent pathname containing git directory; append to git directory's parent
try:
    if commit_only_source :
        subprocess.run(['jupyter', 'nbconvert', '--to', 'script', str(os.environ.get('GIT_PARENT_DIR') + "/" + os.environ.get('GIT_REPO_NAME') + filename)])
        filename = str(os.environ.get('GIT_PARENT_DIR') + "/" + os.environ.get('GIT_REPO_NAME') + filename.replace('ipynb', 'py'))

    print(repo.git.add(str(os.environ.get('GIT_PARENT_DIR') + "/" + os.environ.get('GIT_REPO_NAME') + filename)))
    print(repo.git.commit( a=False, m="{}\n\nUpdated {}".format(msg, filename) ))

except GitCommandError as e:
    print(e)
    self.error_and_return(cwd, "Could not commit changes to notebook: {}".format(git_dir_parent + filename))
    return

# create or switch to remote
try:
    remote = repo.create_remote(git_remote, git_url)
except GitCommandError:
    print("Remote {} already exists...".format(git_remote))
    remote = repo.remote(git_remote)

# push changes
try:
    pushed = remote.push(git_branch)
    assert len(pushed)>0
    assert pushed[0].flags in [git.remote.PushInfo.UP_TO_DATE, git.remote.PushInfo.FAST_FORWARD, git.remote.PushInfo.NEW_HEAD, git.remote.PushInfo.NEW_TAG]
except GitCommandError as e:
    print(e)
    self.error_and_return(cwd, "Could not push to remote {}".format(git_remote))
    return
except AssertionError as e:
    self.error_and_return(cwd, "Could not push to remote {}: {}".format(git_remote, pushed[0].summary))
    return

# open pull request
try:
  github_url = "https://api.github.com/repos/{}/pulls".format(git_repo_upstream)
  github_pr = {
      "title":"{} Notebooks".format(git_user),
      "body":"IPython notebooks submitted by {}".format(git_user),
      "head":"{}:{}".format(git_user, git_remote),
      "base":"master"
  }
  github_headers = {"Authorization": "token {}".format(git_access_token)}
  r = requests.post(github_url, data=json.dumps(github_pr), headers=github_headers)
  if r.status_code != 201:
    print("Error submitting Pull Request to {}".format(git_repo_upstream))
except:
    print("Error submitting Pull Request to {}".format(git_repo_upstream))

# return to directory
os.chdir(cwd)

# close connection
self.write({'status': 200, 'statusText': 'Success!  Changes to {} captured on branch {} at {}'.format(filename, git_branch, git_url)})


def setup_handlers(nbapp):
route_pattern = ujoin(nbapp.settings['base_url'], '/git/commit')
nbapp.add_handlers('.*', [(route_pattern, GitCommitHandler)])



IndentationError: expected an indented block (<ipython-input-32-0898e432df54>, line 96)

In [41]:
git_dir_parent = os.path.dirname(git_dir)
git_dir_parent

'/wynton/home/huang/emaad/Notebooks'

In [36]:
filename

NameError: name 'filename' is not defined

In [40]:
import json
data = json.loads(self.request.body.decode('utf-8'))
filename = urllib.parse.unquote(data['filename'])

NameError: name 'self' is not defined

In [50]:
from urllib.parse import urlparse

urlparse("http://localhost:8892/notebooks/Deep-Learning-Denoising/Deep-Notebook.ipynb")

ParseResult(scheme='http', netloc='localhost:8892', path='/notebooks/Deep-Learning-Denoising/Deep-Notebook.ipynb', params='', query='', fragment='')